# Practice 3: Group Analysis

## Objetives:
1. Group analysis by gender and contract type
2. Report moderation hypotheses tests and report direct and indirect effects
    1. Gender Hypotheses:
        1. H5a: The influence of technical quality on satisfaction is more positive among women than among men
        2. H5b: The influence of financial quality on satisfaction is more positive among women than among men.
        3. H5c: The influence of service quality on satisfaction is more positive among women than among men.
	    4. H5d: The influence of satisfaction on loyalty is more positive among women than among men.
    2. Contract Type Hypotheses:
        1. H6a: The influence of technical quality on satisfaction is more positive among consumers who use postpaid plans than those who use prepaid plans.
	    2. H6b: The influence of financial quality on satisfaction is more positive among consumers who use postpaid plans than those who use prepaid plans.
	    3. H6c: The influence of service quality on satisfaction is more positive among consumers who use postpaid plans than those who use prepaid plans.
	    4. H6d: The influence of satisfaction on loyalty is more positive among consumers who use postpaid plans than those who use prepaid plans.


## Data Preparation

In [22]:
import pandas as pd
import pyreadstat

raw_data_df, meta = pyreadstat.read_sav("../data/EXERCICIO_2025.sav")
raw_data_df.head()

,CASO,f1,f2,f3,f4,f5,f6,f7,P1.1_QA,P1.2_QA,...,P7,P14,ltv2,tempo,clsocial,SATIS,LEAL,ATEND,FIN,TEC
0,NaN,1.0,3.0,6.0,1.0,1.0,50.0,18.0,8.0,6.0,...,3.0,2.0,1373.0,61.0,5.0,6.50,6.2,7.2,9.333333,8.50
1,NaN,1.0,1.0,5.0,1.0,1.0,36.0,35.0,8.0,7.0,...,1.0,1.0,1002.0,47.0,5.0,5.50,4.4,6.8,6.666667,8.00
2,NaN,1.0,3.0,5.0,1.0,1.0,72.0,73.0,10.0,10.0,...,2.0,2.0,2097.0,83.0,5.0,7.50,8.0,9.6,10.000000,8.00
3,NaN,1.0,3.0,9.0,1.0,1.0,24.0,27.0,6.0,6.0,...,1.0,2.0,802.0,35.0,4.0,6.25,6.2,6.4,7.666667,6.50
4,NaN,1.0,4.0,9.0,1.0,1.0,72.0,24.0,2.0,5.0,...,2.0,1.0,4835.0,78.0,4.0,5.25,4.4,4.4,5.666667,5.75


In [23]:
columns_to_filter = [
    'P1.1_QA', 'P1.2_QA', 'P1.3_QA', 'P1.4_QA', 'P1.5_QA', 'P1.15_QA', 'P1.16_QA', 'P1.17_QA',
    'P1.6_QC', 'P1.7_QC', 'P1.8_QC', 'P1.9_QC',
    'P1.10_QT', 'P1.11_QT', 'P1.12_QT', 'P1.13_QT', 'P1.14_QT',
    'P3.1_S', 'P3.2_S', 'P3.3_S', 'P3.4_S',
    'P6.1_L', 'P6.2_L', 'P6.3_L', 'P6.4_L', 'P6.5_L', 'P6.6_L',
    'P14'
]

clean_df = raw_data_df[columns_to_filter].copy()

clean_df.rename(columns=lambda c: c.replace('.', '_'), inplace=True)
clean_df.rename(columns=lambda c: c.replace('C', 'F'), inplace=True)
clean_df.rename(columns=lambda c: c.replace('A', 'S'), inplace=True)
print(raw_data_df.shape)
print(clean_df.shape)
print(clean_df.isnull().sum())

(493, 45)
(493, 28)
P1_1_QS     0
P1_2_QS     0
P1_3_QS     0
P1_4_QS     0
P1_5_QS     0
P1_15_QS    0
P1_16_QS    0
P1_17_QS    0
P1_6_QF     0
P1_7_QF     0
P1_8_QF     0
P1_9_QF     0
P1_10_QT    0
P1_11_QT    0
P1_12_QT    0
P1_13_QT    0
P1_14_QT    0
P3_1_S      0
P3_2_S      0
P3_3_S      0
P3_4_S      0
P6_1_L      0
P6_2_L      0
P6_3_L      0
P6_4_L      0
P6_5_L      0
P6_6_L      0
P14         0
dtype: int64


In [24]:
male_df = clean_df[clean_df['P14'] == 2].copy()
female_df = clean_df[clean_df['P14'] == 1].copy()
print(male_df.shape)
print(female_df.shape)

(232, 28)
(261, 28)


## Gender Models

In [25]:
# from semopy.multigroup import multigroup
# model_description =  """
# QS =~ P1_1_QS + P1_2_QS + P1_4_QS + P1_5_QS + P1_16_QS
# QF =~ P1_6_QF + P1_7_QF + P1_8_QF
# QT =~ P1_10_QT + P1_11_QT + P1_12_QT + P1_13_QT
# S =~ P3_1_S + P3_2_S + P3_3_S + P3_4_S
# L =~ P6_1_L + P6_2_L + P6_3_L + P6_4_L
#
# S ~ QS + QF + QT
# L ~ S
# """
# res = multigroup(
#     model_description,
#     clean_df,
#     group='Gender',
# )
# print(res)

### Modeling

In [26]:
model_description = """
QS =~ P1_1_QS + P1_2_QS + P1_4_QS + P1_5_QS + P1_16_QS
QF =~ P1_6_QF + P1_7_QF + P1_8_QF
QT =~ P1_10_QT + P1_11_QT + P1_12_QT + P1_13_QT
S =~ P3_1_S + P3_2_S + P3_3_S + P3_4_S
L =~ P6_1_L + P6_2_L + P6_3_L + P6_4_L

S ~ QS + QF + QT
L ~ S
"""

#### Male

In [27]:
from semopy import Model
male_model = Model(model_description)
print(male_model.fit(male_df))

Name of objective: MLW
Optimization method: SLSQP
Optimization successful.
Optimization terminated successfully
Objective value: 1.807
Number of iterations: 50
Params: 0.793 1.123 1.085 0.776 1.642 1.666 1.278 1.042 1.067 0.832 0.996 0.987 1.138 0.704 0.984 0.637 0.024 0.262 0.610 0.906 2.063 0.545 0.859 1.661 2.422 1.547 1.971 2.571 2.084 1.730 1.198 2.791 0.564 0.631 0.536 0.919 0.846 1.294 2.462 0.570 0.892 0.675 2.588 0.830 1.108 2.074 1.087


#### Female

In [28]:
female_model = Model(model_description)
print(female_model.fit(female_df))

Name of objective: MLW
Optimization method: SLSQP
Optimization successful.
Optimization terminated successfully
Objective value: 1.508
Number of iterations: 46
Params: 0.856 1.116 1.141 0.677 1.221 1.011 1.166 1.016 0.902 0.906 1.085 1.105 1.087 0.679 0.955 0.373 0.379 0.102 0.626 1.463 2.726 0.930 1.172 1.498 2.293 1.207 1.634 2.128 1.444 1.530 1.367 3.359 0.787 0.699 0.508 0.797 0.672 1.158 2.807 0.609 1.797 1.050 2.894 1.048 1.186 2.074 0.987


### Direct Effects

#### Male

In [29]:
male_raw_item_estimates = (
    male_model.inspect(std_est=True)
         .query("op == '~' and not lval.str.startswith('P')")
)
male_direct_effects_df = (
    male_raw_item_estimates
    .loc[:, ['lval','rval', 'Estimate','Std. Err', 'p-value', 'z-value']]
    .rename(columns={
        'lval': 'Outcome',
        'rval': 'Predictor',
        'Estimate': 'beta',
        'Std. Err': 'SE',
        'p-value': 'p-value',
        'z-value': 't-value'
    })
    .sort_values(by='beta')
)
male_direct_effects_df['hypothesis_result'] = male_direct_effects_df.apply(
    lambda row: 'Supported' if row['p-value'] < 0.05 else 'Not Supported',
    axis=1
)
male_direct_effects_df


,Outcome,Predictor,beta,SE,p-value,t-value,hypothesis_result
1,S,QF,0.024302,0.13024,0.851981,0.186591,Not Supported
2,S,QT,0.261745,0.074415,0.000436,3.517361,Supported
3,L,S,0.610031,0.054598,0.0,11.173243,Supported
0,S,QS,0.636660,0.080442,0.0,7.914545,Supported


#### Female

In [30]:
female_raw_item_estimates = (
    female_model.inspect(std_est=True)
         .query("op == '~' and not lval.str.startswith('P')")
)
female_direct_effects_df = (
    female_raw_item_estimates
    .loc[:, ['lval','rval', 'Estimate', 'Std. Err', 'p-value', 'z-value']]
    .rename(columns={
        'lval': 'Outcome',
        'rval': 'Predictor',
        'Estimate': 'beta',
        'Std. Err': 'SE',
        'p-value': 'p-value',
        'z-value': 't-value'
    })
    .sort_values(by='beta')
)
female_direct_effects_df['hypothesis_result'] = female_direct_effects_df.apply(
    lambda row: 'Supported' if row['p-value'] < 0.05 else 'Not Supported',
    axis=1
)
female_direct_effects_df


,Outcome,Predictor,beta,SE,p-value,t-value,hypothesis_result
2,S,QT,0.101959,0.070445,0.147797,1.447358,Not Supported
0,S,QS,0.372888,0.056994,0.0,6.542589,Supported
1,S,QF,0.379274,0.082745,0.000005,4.583658,Supported
3,L,S,0.625585,0.06514,0.0,9.603669,Supported


#### Wald statistic

$Z = \frac{\beta_1 - \beta_2}{\sqrt{SE_1^2 + SE_2^2}}$
Clogg, C. C., Petkova, E., & Haritou, A. (1995). Statistical methods for comparing regression coefficients between models. American Journal of Sociology, 100(5), 1261–1293. https://doi.org/10.1086/230638

Please refer to [Wald test](https://en.wikipedia.org/wiki/Wald_test) and [Testing equality of coefficients from two different regressions](https://stats.stackexchange.com/questions/93540/testing-equality-of-coefficients-from-two-different-regressions?utm_source=chatgpt.com) for more information.

In [31]:
from scipy.stats import norm

def direct_wald_test(beta1, se1, beta2, se2):
    z = (beta1 - beta2) / ((se1**2 + se2**2)**0.5)
    p = 2 * (1 - norm.cdf(abs(z)))
    return z, p

In [32]:
female_df = female_direct_effects_df.copy()
male_df = male_direct_effects_df.copy()

female_df = female_df.rename(columns={
    'beta': 'beta_f',
    't-value': 't_f',
    'p-value': 'p_f',
    'SE': 'se_f'
})

male_df = male_df.rename(columns={
    'beta': 'beta_m',
    't-value': 't_m',
    'p-value': 'p_m',
    'SE': 'se_m'
})

In [33]:
merged_df = pd.merge(
    female_df[['Outcome', 'Predictor', 'beta_f', 'se_f']],
    male_df[['Outcome', 'Predictor', 'beta_m', 'se_m']],
    on=['Outcome', 'Predictor']
)

In [34]:
merged_df[['z_wald', 'p_wald']] = merged_df.apply(
    lambda row: direct_wald_test(
        row['beta_f'], row['se_f'], row['beta_m'], row['se_m']
    ),
    axis=1, result_type='expand'
)

In [35]:
merged_df['difference_significant'] = merged_df['p_wald'].apply(lambda p: 'Yes' if p < 0.05 else 'No')
merged_df

,Outcome,Predictor,beta_f,se_f,beta_m,se_m,z_wald,p_wald,difference_significant
0,S,QT,0.101959,0.070445,0.261745,0.074415,-1.559346,0.118915,No
1,S,QS,0.372888,0.056994,0.636660,0.080442,-2.675557,0.007461,Yes
2,S,QF,0.379274,0.082745,0.024302,0.13024,2.300505,0.021420,Yes
3,L,S,0.625585,0.06514,0.610031,0.054598,0.182989,0.854807,No


### Indirect Effects

#### Male

##### QA -> L, QF -> L, QT -> L through S

In [36]:
male_standardised_paths = (
    male_model.inspect(std_est=True, se_robust=True)
         .query("op == '~'")
         .set_index(['lval', 'rval'])
)

##### Mediator-to-outcome path  (S -> L)

In [37]:
male_b_path_coefficient = float(
    male_standardised_paths.loc[('L', 'S'), 'Est. Std']
)
male_b_path_standard_error = float(
    male_standardised_paths.loc[('L', 'S'), 'Std. Err']
)

##### Loop over each quality dimension (predictor-to-mediator paths)

In [38]:
import math
male_results = []
for predictor in ['QS', 'QF', 'QT']:
    male_a_path_coefficient = float(
        male_standardised_paths.loc[('S', predictor), 'Est. Std']
    )
    male_a_path_standard_error = float(
        male_standardised_paths.loc[('S', predictor), 'Std. Err']
    )

    # Indirect effect = a * b
    male_indirect_effect = male_a_path_coefficient * male_b_path_coefficient

    # Sobel standard error: sqrt(b^2 * SE_a^2 + a^2 * SE_b^2)
    male_indirect_se = math.sqrt(
        (male_b_path_coefficient ** 2) * (male_a_path_standard_error ** 2) +
        (male_a_path_coefficient ** 2) * (male_b_path_standard_error ** 2)
    )

    male_z_value = male_indirect_effect / male_indirect_se
    # Two-tailed p-value from z
    male_p_value = math.erfc(abs(male_z_value) / math.sqrt(2))

    male_results.append({
        'Path': f'{predictor} -> L (via S)',
        'Indirect Effect': round(male_indirect_effect, 3),
        'Standard Error':  round(male_indirect_se, 3),
        'z-value':         round(male_z_value, 2),
        'p-value':         round(male_p_value, 4)
    })

In [39]:
male_indirect_effects_df = pd.DataFrame(male_results)
male_indirect_effects_df

,Path,Indirect Effect,Standard Error,z-value,p-value
0,QS -> L (via S),0.453,0.076,5.94,0.0000
1,QF -> L (via S),0.010,0.111,0.09,0.9270
2,QT -> L (via S),0.167,0.058,2.85,0.0043


#### Female

##### QA -> L, QF -> L, QT -> L through S

In [40]:
female_standardised_paths = (
    female_model.inspect(std_est=True, se_robust=True)
         .query("op == '~'")
         .set_index(['lval', 'rval'])
)

##### Mediator-to-outcome path  (S -> L)

In [41]:
female_b_path_coefficient = float(
    female_standardised_paths.loc[('L', 'S'), 'Est. Std']
)
female_b_path_standard_error = float(
    female_standardised_paths.loc[('L', 'S'), 'Std. Err']
)

##### Loop over each quality dimension (predictor-to-mediator paths)

In [42]:
import math
female_results = []
for predictor in ['QS', 'QF', 'QT']:
    female_a_path_coefficient = float(
        female_standardised_paths.loc[('S', predictor), 'Est. Std']
    )
    female_a_path_standard_error = float(
        female_standardised_paths.loc[('S', predictor), 'Std. Err']
    )

    # Indirect effect = a * b
    female_indirect_effect = female_a_path_coefficient * female_b_path_coefficient

    # Sobel standard error: sqrt(b^2 * SE_a^2 + a^2 * SE_b^2)
    female_indirect_se = math.sqrt(
        (female_b_path_coefficient ** 2) * (female_a_path_standard_error ** 2) +
        (female_a_path_coefficient ** 2) * (female_b_path_standard_error ** 2)
    )

    female_z_value = female_indirect_effect / female_indirect_se
    # Two-tailed p-value from z
    female_p_value = math.erfc(abs(female_z_value) / math.sqrt(2))

    female_results.append({
        'Path': f'{predictor} -> L (via S)',
        'Indirect Effect': round(female_indirect_effect, 3),
        'Standard Error':  round(female_indirect_se, 3),
        'z-value':         round(female_z_value, 2),
        'p-value':         round(female_p_value, 4)
    })

In [43]:
female_indirect_effects_df = pd.DataFrame(female_results)
female_indirect_effects_df

,Path,Indirect Effect,Standard Error,z-value,p-value
0,QS -> L (via S),0.262,0.054,4.83,0.0000
1,QF -> L (via S),0.210,0.067,3.11,0.0019
2,QT -> L (via S),0.061,0.045,1.36,0.1751
